In [ ]:
#deal with one stock first
one_stock = yf.download(['CVS'], start = "2005-01-01", end = "2015-01-01")
df = one_stock[['Close']]

df['log_return'] = df['Close'].div(df['Close'].shift(1)).apply(np.log)
df['log_return'].sum()

para_days=30
for i in range(para_days):
    df['log_return'+str(i)] = df['log_return'].shift(i)
    
    ##can add volumn here if needed
    
df.head()

df['30days_return'] = df['log_return'].rolling(para_days).sum()

df['target'] = df[str(para_days)+'days_return'].shift(-para_days)
df.head(3)

df=df.dropna()
df.head(3)

df=df.drop(['Close', '30days_return','log_return'], axis=1)

df.head(3)

#keep in mind that log_return3 means the log return of 3 days ago

data_np = np.array(df)[:,:-1]
data_np
targets_np=np.array(df)[:,-1]

targets_np.shape, data_np.shape

normalization_factors = {'mean': targets_np.mean(), 'std':targets_np.std()}
normalization_factors

targets_np_normalized = (targets_np-normalization_factors['mean'])/normalization_factors['std']
targets_np_normalized

data_np = data_np[:,::-1]
data_np

#to be safe, we use the first 80% of data as training, 10% validation, last 20% for testing
def data_split(data_np, targets_np):
    assert(len(data_np) == len(targets_np))
    n = len(data_np)
    n1 = int(n*0.7)
    n2 = int(n*0.1)
    data_train = data_np[:n1]
    data_val = data_np[n1:n2]
    data_test=data_np[n2:]
    targets_train=targets_np[:n1]
    targets_val=targets_np[n1:n2]
    targets_test=targets_np[n2:]
    return {'train':{'data':data_train, 'targets':targets_train}, 
           'val':{'data':data_val, 'targets':targets_val},
           'test':{'data':data_test, 'targets':targets_test}}

dic = data_split(data_np, targets_np)
dic['train']['data'].shape, dic['train']['targets'].shape

# Now we build a more complicated ML algorithm, as follows:
# 1. use 30 days close data to predict the increment after a month
# 2. for each 30 days we have several inputs:
#     a. 30 day log return
#     b. 30 days split into 6 periods * 5 days log return of all stocks
#     c. 30 days log return with log weight
#     d. 30 days volatility, say 6 * 5 days
    